In [1]:
import dr_util.file_utils as fu

In [2]:
# Load data
roam_data = fu.load_file("/Users/daniellerothermel/Desktop/drotherm_roam.json")
len(roam_data)

**Goal:** Programatically extract, surface and annotate useful things in work roam graph data.

## Define Clases & Utils

In [155]:
class RoamNode:
    def __init__(self, json, start_depth=0):
        self.refs_uid_blacklist = set([
            'KVGudD7AP', # [[DONE]]
            'e2rS3SVH7', # [[TODO]]
        ])
        if json is None:
            raise Exception("None json page")
        if not isinstance(json, dict):
            raise Exception(
                "RoamNode expects single page json"
            )
            
        self.raw_data = None
        self.depth = start_depth

        self.title = None
        self.string = None
        self.uid = None
        self.create_time = None
        self.edit_time = None
        self.children = []
        self.refs = []
        self.other_keys = {}
        
        self.import_json(json)

    def __repr__(self):
        indent = "  " * self.depth
        if self.title is not None:
            rep_str = f"{self.title}\n"
            rep_str += f" {indent} {self.uid=} {self.refs=}\n"
        else:
            rep_str = f"{indent} - {self.string}"
            if len(self.refs) > 0:
                rep_str += f"\n{indent} ==> {self.uid=} {self.refs=}\n"
        #rep_str += (
        #    f" {indent} - {self.depth=}\n"
        #    f" {indent} - {self.uid=}\n"
        #    f" {indent} - {self.create_time=}\n"
        #    f" {indent} - {self.edit_time=}\n"
        #    f" {indent} - {len(self.children)=}\n"
        #    f" {indent} - {self.refs=}\n"
        #    f" {indent} - {self.other_keys.keys()}\n"
        #)
        return rep_str

    def import_json(self, json):
        self.raw_data =  json

        for k, v in json.items():
            if k[0] == ':' or "user" in k:
                continue

            # TODO: cleanup
            if k == 'title':
                self.title = v
            elif k == 'string':
                self.string = v
            elif k == 'uid':
                self.uid = v
            elif k == 'create-time':
                self.create_time = v
            elif k == 'edit-time':
                self.edit_time = v
            elif k == 'children':
                for ch in v:
                    self.children.append(
                        RoamNode(ch, start_depth=self.depth+1)
                    )
            elif k == 'refs':
                for vv in v:
                    if 'uid' in vv and vv['uid'] not in self.refs_uid_blacklist:
                        self.refs.append(vv)
            else:
                self.other_keys[k] = v

            

In [156]:
def print_roam_node(rn):
    node_i = 0
    nodes = [rn]
    while node_i < len(nodes):
        print(nodes[node_i])
        for ch in nodes[node_i].children:
            nodes.append(ch)
        node_i += 1

---

## Examples

### Example of Daily Page

In [163]:
print_roam_node(RoamNode(roam_data[0]))

October 8th, 2020
  self.uid='10-08-2020' self.refs=[]

   - **Thursday**
   - Today I
     - Made my RR account
     - Watched: Attention is All You Need
     - Setup: Ramping up on LMs note
     - Watched the first two videos (intro to transfer learning and 🤗, lstm is dead)


### Example of New Paper Page Format

In [171]:
print_roam_node(RoamNode(roam_data[2000]))

2019| Solving Rubik's Cube with a Robot Hand
  self.uid='Jb7PatuK4' self.refs=[]

   - Paper: [arxiv](https://arxiv.org/abs/1910.07113)
   - Authors: #OpenAI
   ==> self.uid='RVlI9Mtgs' self.refs=[{'uid': '3zYNzVFy8'}]

   - #[[Domain Randomization]] #[[Curriculum Learning]] #Sim2Real #[[Transfer Learning]] #[[[[Generalization]] in [[RL]]]]
   ==> self.uid='l1UBICZk9' self.refs=[{'uid': 'q59cNnGpW'}, {'uid': '_4e_SPqLS'}, {'uid': 'Gnb9U6cJl'}, {'uid': '1_QMw77kQ'}, {'uid': 'R5SeictTQ'}, {'uid': 'eaCGvDycx'}, {'uid': 'yNs5rNiHf'}]



### Example of Old Paper Page Format

In [160]:
print_roam_node(RoamNode(roam_data[2]))

Attention Is All You Need
  self.uid='PshQdKxeS' self.refs=[]

   - **Authors**: #[[Ashish Vaswani]], #[[Noam Shazeer]], #[[Jakob Uszkoreit]], #[[Llion Jones]], #[[Aidan N. Gomez]], #[[Lukasz Kaiser]], #[[Illia Polosukhin]]
**Organization**: #Google #Papers2017
**Topics**: #paperssorted #ToAnki #Transformers #NLP
**Paper Link**: https://arxiv.org/abs/1706.03762
**Video Link**: https://www.youtube.com/watch?v=iDulhoQ2pro&t=11s
   ==> self.uid='CfOF1srRm' self.refs=[{'uid': 'LKqQsFmuk'}, {'uid': '3qZcbQeaM'}, {'uid': '0LcSf-pj6'}, {'uid': 'U8xh6WGvP'}, {'uid': 'wUGVJUtK8'}, {'uid': 'h8FxKBCFc'}, {'uid': 'fq2sVAtYR'}, {'uid': 'zUEH7Q5At'}, {'uid': 'a43hzItKe'}, {'uid': 'yrANQ6m5o'}, {'uid': 'sWHx_W9Xx'}, {'uid': 'LN_hZEpob'}, {'uid': 'oVmuQcd4q'}]

   - ----------------------------------
   - Processing Stages
   - ----------------------------------
   - **Summary**
   - **Synthesized Detailed Notes**
     - {{[[DONE]]}} Easy Notes
     - {{[[DONE]]}} Synthesize Easy Notes
     - {{[[DONE